In [1]:
import vk
import pandas as pd
import time
from datetime import datetime

In [2]:
# opennig vk session with access token.
session = vk.Session(access_token='5951a0707374abf660b6aae5bbbc5df47d240d1641765ddd07f681a3de8a8ce7ba1f404c645c58d365a24')
api = vk.API(session)
# example use with id, my id
api.users.get(user_ids='architectofreality' )

[{'first_name': 'Alexander', 'last_name': 'Ovchinnikov', 'uid': 182927367}]

In [3]:
#searching for a keyword glaukoma starting 48 hours ago, max 200 counts can be scrapped at once
glaukoma=api.newsfeed.search(q='glaukoma',count=200, start_time=48)

### get last 200 posts --> take the time of the last post --> at the next iteration set the end time = time_last_post-1. Set the start time to 1 

In [4]:
%%time
glaukoma_rus=api.newsfeed.search(q='glaukoma',count=200, end_time=1512378120, start_time=1512378121-86400)

Wall time: 127 ms


In [5]:
# dropping the first element and creat a dataframe with the rest
df=pd.DataFrame(glaukoma_rus[1:])

In [6]:
#getting info of each user, deactivated column
ids=pd.DataFrame(api.users.get(user_ids=abs(df['owner_id'])))
ids.head()

,deactivated,first_name,last_name,uid
0,deleted,Elena,Zatylkina,70740559


In [7]:
%%time
desired_time = 86400*1000 # how much back we want to go in seconds
date = time.time() # now
excess_time = True
all_df = pd.DataFrame()
c=1
while excess_time:
     # print iteration count, extended tells us group vs user
    g=api.newsfeed.search(q=['глаз'],count=200, end_time = date-1, start_time=1,extended=1,
                          latitude = 54.70649, longitude = 20.51095)
    if len(g)>1: # check if g contains data
        all_df = pd.concat([all_df,pd.DataFrame(g[1:])])
        date=g[-1]['date']
    elif len(g) <= 1 and c == 1: # check if there is data
        print('No posts with that keyword(s)!')
        break
    elif len(g) <= 1 and c != 1: # check if there are posts eft to scrape
        print('No more posts to scrape!')
        break
    
    if date<time.time()-desired_time: # check termination condition
        excess_time = False
    print(c,'iteration')
    c+=1

1 iteration
2 iteration
3 iteration
4 iteration
Wall time: 7.86 s


In [8]:
all_df.head()

,attachment,attachments,comments,date,from_id,geo,group,id,likes,marked_as_ads,owner_id,post_source,post_type,reposts,signer_id,text,user
0,"{'type': 'photo', 'photo': {'pid': 456240198, ...","[{'type': 'photo', 'photo': {'pid': 456240198,...","{'count': 0, 'groups_can_post': True, 'can_pos...",1514199488,21581188,"{'type': 'point', 'coordinates': '54.718054370...",NaN,5141,"{'count': 5, 'user_likes': 0, 'can_like': 1, '...",NaN,21581188,{'type': 'vk'},post,"{'count': 0, 'user_reposted': 0}",NaN,Рубрика глаз продолжается. Вчера посмотрела ви...,"{'uid': 21581188, 'first_name': 'Kristina', 'l..."
1,"{'type': 'photo', 'photo': {'pid': 456240981, ...","[{'type': 'photo', 'photo': {'pid': 456240981,...","{'count': 0, 'groups_can_post': True, 'can_pos...",1514128618,26019602,"{'type': 'place', 'coordinates': '54.716708452...",NaN,8723,"{'count': 29, 'user_likes': 0, 'can_like': 1, ...",NaN,26019602,"{'type': 'api', 'platform': 'iphone'}",post,"{'count': 0, 'user_reposted': 0}",NaN,— Так в чем счастье? <br>— Счастье? Видеть в г...,"{'uid': 26019602, 'first_name': 'Tatyana', 'la..."
2,"{'type': 'photo', 'photo': {'pid': 456245488, ...","[{'type': 'photo', 'photo': {'pid': 456245488,...","{'count': 0, 'groups_can_post': True, 'can_pos...",1514116852,20217679,"{'type': 'point', 'coordinates': '54.708500782...",NaN,6727,"{'count': 3, 'user_likes': 0, 'can_like': 1, '...",NaN,20217679,"{'type': 'api', 'platform': 'instagram', 'url'...",post,"{'count': 0, 'user_reposted': 0}",NaN,Эх☺️Увидеть Париж-умереть-возродиться-снова ув...,"{'uid': 20217679, 'first_name': 'Nikita', 'las..."
3,"{'type': 'photo', 'photo': {'pid': 456241867, ...","[{'type': 'photo', 'photo': {'pid': 456241867,...","{'count': 0, 'groups_can_post': True, 'can_pos...",1513958220,109293854,"{'type': 'point', 'coordinates': '54.696070746...",NaN,7445,"{'count': 32, 'user_likes': 0, 'can_like': 1, ...",NaN,109293854,"{'type': 'api', 'platform': 'instagram', 'url'...",post,"{'count': 0, 'user_reposted': 0}",NaN,Не только же коричневые цвета на глазах делать...,"{'uid': 109293854, 'first_name': 'Maria', 'las..."
4,"{'type': 'photo', 'photo': {'pid': 456239341, ...","[{'type': 'photo', 'photo': {'pid': 456239341,...","{'count': 0, 'groups_can_post': True, 'can_pos...",1513957008,314271,"{'type': 'point', 'coordinates': '54.719526391...",NaN,8177,"{'count': 7, 'user_likes': 0, 'can_like': 1, '...",NaN,314271,"{'type': 'api', 'platform': 'instagram', 'url'...",post,"{'count': 0, 'user_reposted': 0}",NaN,"Друзья, наконец - то это свершилось! Рассказы...","{'uid': 314271, 'first_name': 'Dima', 'last_na..."


In [9]:
# take all positive ids ( user ids , negative ids are group ids)
all_df_new=all_df[all_df.owner_id>0] 

# get all users
all_id=pd.DataFrame(api.users.get(user_ids=all_df_new['owner_id']))

# fill active users status with valid
all_id.fillna('Valid',inplace=True) 
try:    
    # select all deactivated user ids
    deactivated_ids = list(all_id[all_id['deactivated']!='Valid'].uid)

    # select all active user posts
    active_users = all_df_new[-all_df_new.owner_id.isin(deactivated_ids)] 
except:
    active_users = all_df_new  

In [10]:
# creating a dataframe for dictionary values
from tqdm import tqdm
new_df = pd.DataFrame(columns = ['user_id','post','sex','Name_Surname',
                                 'likes','comments','reposts','platform','date'])
# resetting the index it make circles of 0-200
active_users.reset_index(inplace = True)
for i in tqdm(range(len(active_users))):    
    new_df.loc[i,'user_id'] = active_users.user.loc[i]['uid']
    new_df.loc[i,'sex'] = active_users.user.loc[i]['sex']   
    new_df.loc[i,'Name_Surname'] = active_users.user.loc[i]['first_name']+'_'+active_users.user.loc[i]['last_name']
    new_df.loc[i,'post'] = active_users.text.loc[i]
    new_df.loc[i,'likes'] = active_users.likes.loc[i]['count']
    new_df.loc[i,'comments'] = active_users.comments.loc[i]['count'] 
    new_df.loc[i,'reposts'] = active_users.reposts.loc[i]['count']
    try:
        new_df.loc[i,'platform'] = active_users.post_source.loc[i]['type']+'_'+active_users.post_source.loc[i]['platform']
    except:
        new_df.loc[i,'platform'] = active_users.post_source.loc[i]['type']
    new_df.loc[i,'date'] = datetime.fromtimestamp(active_users.date.loc[i]).strftime('%Y-%m-%d %H:%M:%S')
new_df['sex'] = new_df['sex'].apply(lambda x: 'Female' if x==1 else 'Male' if x==2  else 'Not Specified')
new_df.head()

100%|███████████████████████████████████████████████████████████████████████████████| 762/762 [00:03<00:00, 208.08it/s]


,user_id,post,sex,Name_Surname,likes,comments,reposts,platform,date
0,21581188,Рубрика глаз продолжается. Вчера посмотрела ви...,Female,Kristina_Chernenko,5,0,0,vk,2017-12-25 14:58:08
1,26019602,— Так в чем счастье? <br>— Счастье? Видеть в г...,Female,Tatyana_Maslakova,29,0,0,api_iphone,2017-12-24 19:16:58
2,20217679,Эх☺️Увидеть Париж-умереть-возродиться-снова ув...,Male,Nikita_Yolkin,3,0,0,api_instagram,2017-12-24 16:00:52
3,109293854,Не только же коричневые цвета на глазах делать...,Female,Maria_Samatova,32,0,0,api_instagram,2017-12-22 19:57:00
4,314271,"Друзья, наконец - то это свершилось! Рассказы...",Male,Dima_Selin,7,0,0,api_instagram,2017-12-22 19:36:48


In [11]:
short_df =new_df.copy()

In [12]:
# remove posts with more than 300 characters and posts with no characters
short_df['post']=short_df.post.apply(lambda x: x if len(x)<300 and len(x) !=0 else 'Too Long')

In [13]:
short_df[short_df['post']!='Too Long'].head()

,user_id,post,sex,Name_Surname,likes,comments,reposts,platform,date
1,26019602,— Так в чем счастье? <br>— Счастье? Видеть в г...,Female,Tatyana_Maslakova,29,0,0,api_iphone,2017-12-24 19:16:58
2,20217679,Эх☺️Увидеть Париж-умереть-возродиться-снова ув...,Male,Nikita_Yolkin,3,0,0,api_instagram,2017-12-24 16:00:52
3,109293854,Не только же коричневые цвета на глазах делать...,Female,Maria_Samatova,32,0,0,api_instagram,2017-12-22 19:57:00
8,178178020,#новогоднийвенок радует глаз . Моя медитация о...,Female,Irina_Tatyankina,7,0,0,api_instagram,2017-12-19 13:31:01
10,21581188,Шикарные зеленые глаза! 👁👁 И собственно класси...,Female,Kristina_Chernenko,11,0,0,vk,2017-12-17 14:26:00
